In [ ]:
#!pip uninstall tensorflow

In [ ]:
#!pip install tensorflow-gpu==2.0.0

In [43]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras import layers
print(tf.__version__)
import time
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.patches as mpatches
from sklearn.metrics import roc_auc_score
import random
import itertools
from matplotlib.patches import Ellipse
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
import tensorflow_addons as tfa


2.4.0


In [44]:
size = 25

In [45]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95
tf.compat.v1.Session(config=config)

![title](擷取.PNG)

In [46]:
def gernate_data(n, nu, seed, range_, multi, size):
    ## normal
    np.random.seed(seed=seed)
    y_train = np.ones((int(n*(1-nu)),))
    mean = np.random.uniform(-5, 5, size)
    cov_  = np.random.uniform(0, 1, (size, size))* 3
    cov = np.dot(cov_.T,cov_)
    x_train = np.random.multivariate_normal(mean, cov, (int(n*(1-nu))))
    x_train = x_train.astype('float32')

    
    
    np.random.seed(seed=seed)
    x_trains_ = None ## normal + outlier
    for i in range(multi):
        mean = np.random.uniform(-range_, range_, size)
        cov_ = np.random.randn(size, size)
        cov = np.dot(cov_.T,cov_)
        x_train_ = np.random.multivariate_normal(mean, cov, (int(n*nu/multi)), check_valid  = 'raise')
        try:
            x_trains_ = np.vstack((x_trains_, x_train_))
        except:
            x_trains_ = x_train_
    ## 畫圖
    '''fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(x_train[:, 0], x_train[:, 1],x_train[:, 2], s=0.9, label = 'normal')
    ax.scatter(x_trains_[:,0], x_trains_[:,1], x_trains_[:,2], s=0.9, label = 'outlier')
    plt.legend()
    plt.title('normal + outlier')
    plt.show()'''

    y_trains_ = np.zeros(((len(x_trains_),)))
    x_train_mix = np.vstack((x_trains_, x_train)).astype('float32')
    y_train_mix = np.hstack((y_trains_, y_train)).astype('float32')
    
    return x_train_mix, y_train_mix

## 參數設定

In [47]:

z_dim = 5
h_dim = [10, 5]
regular = 1e-5
max_lr = 1e-6
base_lr = 1e-5


n_epochs = 1000

optimizer = tf.keras.optimizers.Adam(lr = base_lr)

In [48]:
def make_ae_model():

    inputs = tf.keras.Input(shape=(size,), name='inputs')

    x = tf.keras.layers.Dense(h_dim[0], activation='relu', use_bias=True)(inputs)
    x = tf.keras.layers.Dense(h_dim[1], activation='linear', use_bias=False, name = 'hidden')(x)
    x = tf.keras.layers.Dense(h_dim[0], activation='relu', use_bias=True)(x)
    x = tf.keras.layers.Dense(size, activation='linear', use_bias=True)(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)

    return model

ae_model = make_ae_model()
ae_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 25)]              0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                260       
_________________________________________________________________
hidden (Dense)               (None, 5)                 50        
_________________________________________________________________
dense_9 (Dense)              (None, 10)                60        
_________________________________________________________________
dense_10 (Dense)             (None, 25)                275       
Total params: 645
Trainable params: 645
Non-trainable params: 0
_________________________________________________________________


In [49]:
def make_model(model):
    
    model.trainable = True
    x = model.get_layer('hidden').output
    
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model_out = tf.keras.Model(inputs = model.input, outputs = outputs)
    
    return model_out

model = make_model(ae_model)
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 25)]              0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                260       
_________________________________________________________________
hidden (Dense)               (None, 5)                 50        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 6         
Total params: 316
Trainable params: 316
Non-trainable params: 0
_________________________________________________________________


## Train SA-SVDD

In [50]:
class train_classfication():
    
    def __init__(self , binary_model, train_dataset, nu ):
        
        self.nu = nu
        self.binary_model = binary_model
        self.optimizer = tf.keras.optimizers.Adam(lr = 1e-2, epsilon = 1e-6)
        self.train_dataset = train_dataset

    def loss(self, batch_x, batch_y):
        
        fl = tfa.losses.SigmoidFocalCrossEntropy()
        end_loss = fl(y_true = batch_y, y_pred = batch_x)

        return end_loss

    @tf.function
    def train_model(self, batch_x, batch_y):
        with tf.GradientTape() as tape:
            output = self.binary_model(batch_x, training=True)
            output = tf.squeeze(output)
            output_loss = self.loss(output, batch_y)

        grads = tape.gradient(output_loss, self.binary_model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.binary_model.trainable_variables))

        return output_loss
    
    def train( self, lr_milestone = 15):
        
        
        pre_loss = np.inf
        loss = 0.
        #while abs(loss - pre_loss) > 1e-6:
        for i in range(100):
            epoch_ae_loss_avg = tf.metrics.Mean()

            for batch, (batch_x, batch_y) in enumerate(train_dataset):

                ae_loss = self.train_model(batch_x, batch_y)
                epoch_ae_loss_avg(ae_loss)
            #loss_list.append(epoch_ae_loss_avg.result())
        
        self.binary_model.save_weights('model_{}'.format(
            self.nu))
        
    def score(self, data):
        self.binary_model.load_weights('model_{}' .format(
            self.nu))
        output = self.binary_model(data, training = False)
        return output
    
    def train_result(self, x_novali, y_novali, show_img = False):
        
        test_auc = 0.
        confusionmatrix = 0.
        if show_img:
            scores = self.score(x_novali).numpy()
            test_auc = self.result(scores, y_novali)

            
            
        return radius_end, test_auc, confusionmatrix
    
    def result(self, scores, y_test):
        
        y_true = y_test
        test_auc = roc_auc_score(y_true, scores)

        return test_auc 
    
    def test_result(self, x_test, y_test):
        
        scores = self.score(x_test).numpy()
        test_auc = self.result(scores, y_test)

        return test_auc

In [51]:
nu = 0.1
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_mix, y_train_mix))
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [52]:
result = np.zeros((25, 3))
warm_up_n_epochs = 10
for i in range(25):
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    ae_model.fit(x_train_mix, x_train_mix, epochs = 30, verbose = 0)
    ## 訓練 SVDD
    binary_model = make_model(ae_model)
    binary = train_classfication(binary_model, train_dataset, nu)
    binary.train(x_train_mix)
    auc  = binary.test_result(x_test_mix, y_test_mix)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.924159034792368
epoch =  1 auc =  0.952507519640853
epoch =  2 auc =  0.9361880471380472
epoch =  3 auc =  0.9396383277216611
epoch =  4 auc =  0.9435518518518519
epoch =  5 auc =  0.9403551066217732
epoch =  6 auc =  0.9434894500561168
epoch =  7 auc =  0.9311827721661056
epoch =  8 auc =  0.8976104377104377
epoch =  9 auc =  0.933331088664422
epoch =  10 auc =  0.9411315375982043
epoch =  11 auc =  0.9573205387205387
epoch =  12 auc =  0.9271482042648709
epoch =  13 auc =  0.9220765432098766
epoch =  14 auc =  0.9329731762065095
epoch =  15 auc =  0.9452910213243548
epoch =  16 auc =  0.9439716049382717
epoch =  17 auc =  0.9674991021324354
epoch =  18 auc =  0.9616514029180696
epoch =  19 auc =  0.9474950056116722
epoch =  20 auc =  0.9129679573512907
epoch =  21 auc =  0.93339898989899
epoch =  22 auc =  0.9534141414141415
epoch =  23 auc =  0.9435142536475869
epoch =  24 auc =  0.9345864197530864


In [53]:
np.mean(result, 0)

array([12.        ,  0.93865814, 14.94084741])

In [54]:
np.std(result, 0)

array([7.21110255, 0.01480397, 0.26979742])

In [55]:
nu = 0.2
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_mix, y_train_mix))
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [56]:
result = np.zeros((25, 3))
warm_up_n_epochs = 10
for i in range(25):
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    ae_model.fit(x_train_mix, x_train_mix, epochs = 30, verbose = 0)
    ## 訓練 SVDD
    binary_model = make_model(ae_model)
    binary = train_classfication(binary_model, train_dataset, nu)
    binary.train(x_train_mix)
    auc  = binary.test_result(x_test_mix, y_test_mix)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.9629691102756893
epoch =  1 auc =  0.9634850250626568
epoch =  2 auc =  0.9364810463659148
epoch =  3 auc =  0.9608318609022556
epoch =  4 auc =  0.9693107142857142
epoch =  5 auc =  0.9008003446115288
epoch =  6 auc =  0.9479872180451129
epoch =  7 auc =  0.9777182957393483
epoch =  8 auc =  0.956848088972431
epoch =  9 auc =  0.9107729949874687
epoch =  10 auc =  0.945560902255639
epoch =  11 auc =  0.9597188596491227
epoch =  12 auc =  0.9141996553884713
epoch =  13 auc =  0.9609213032581454
epoch =  14 auc =  0.9332078947368421
epoch =  15 auc =  0.9300077067669174
epoch =  16 auc =  0.9568131578947369
epoch =  17 auc =  0.9167437969924812
epoch =  18 auc =  0.9489448934837093
epoch =  19 auc =  0.9268577380952382
epoch =  20 auc =  0.927665820802005
epoch =  21 auc =  0.9503483082706765
epoch =  22 auc =  0.9395378446115288
epoch =  23 auc =  0.9234465538847118
epoch =  24 auc =  0.9551843358395989


In [57]:
np.mean(result, 0)

array([12.        ,  0.94305454, 14.97168523])

In [58]:
np.std(result, 0)

array([7.21110255, 0.01982106, 0.38492927])

In [59]:
nu = 0.3
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_mix, y_train_mix))
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [60]:
result = np.zeros((25, 3))
warm_up_n_epochs = 10
for i in range(25):
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    ae_model.fit(x_train_mix, x_train_mix, epochs = 30, verbose = 0)
    ## 訓練 SVDD
    binary_model = make_model(ae_model)
    binary = train_classfication(binary_model, train_dataset, nu)
    binary.train(x_train_mix)
    auc  = binary.test_result(x_test_mix, y_test_mix)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.8836340476190475
epoch =  1 auc =  0.9520894047619047
epoch =  2 auc =  0.957779380952381
epoch =  3 auc =  0.9393547380952381
epoch =  4 auc =  0.9623134523809523
epoch =  5 auc =  0.9596931666666667
epoch =  6 auc =  0.9455563095238095
epoch =  7 auc =  0.9410514523809523
epoch =  8 auc =  0.9296282142857144
epoch =  9 auc =  0.9435848095238095
epoch =  10 auc =  0.9361448095238095
epoch =  11 auc =  0.9669114285714285
epoch =  12 auc =  0.9469449523809523
epoch =  13 auc =  0.9469300714285713
epoch =  14 auc =  0.9426777142857143
epoch =  15 auc =  0.9256253333333333
epoch =  16 auc =  0.9364204285714286
epoch =  17 auc =  0.9644617619047617
epoch =  18 auc =  0.9380168095238096
epoch =  19 auc =  0.9417813333333334
epoch =  20 auc =  0.9408916904761904
epoch =  21 auc =  0.9553310000000002
epoch =  22 auc =  0.9476690714285714
epoch =  23 auc =  0.9235424761904762
epoch =  24 auc =  0.9481223333333332


In [61]:
np.mean(result, 0)

array([12.        ,  0.94304625, 15.08144714])

In [62]:
np.std(result, 0)

array([7.21110255, 0.01642539, 0.44931136])